# Creating and Managing Experiments

The last two guides showcased how you can create and run synthetic discussions, and synthetic annotations using LLMs. However, in order to produce robust results for a hypothesis, you may need to produce multiple annotated discussions. 

While this is certainly possible using the `Discussion` and `Annotation` APIs, SynDisco offers a high-level API that automatically creates and manages multiple discussions with different configurations. This guide will showcase how you can leverage this API to automate your experiments.

## Experiments

An`Experiment` is an entity that generates and runs `jobs`. As noted above, you can use Experiments in lieu of jobs. 

## Discussion Experiments

### Setup

In [ ]:
%cd ../../src

In [ ]:
from syndisco.backend.turn_manager import RoundRobbin
from syndisco.backend.actors import LLMActor, ActorType
from syndisco.backend.model import TransformersModel
from syndisco.backend.persona import LLMPersona


CONTEXT = "You are taking part in an online conversation"
INSTRUCTIONS = "Act like a human would"


llm = TransformersModel(
    model_path="unsloth/Llama-3.2-1B-Instruct",
    name="test_model",
    max_out_tokens=100,
)
persona_data = [
    {
        "username": "Emma35",
        "age": 38,
        "sex": "female",
        "education_level": "Bachelor's",
        "sexual_orientation": "Heterosexual",
        "demographic_group": "Latino",
        "current_employment": "Registered Nurse",
        "special_instructions": "",
        "personality_characteristics": [
            "compassionate",
            "patient",
            "diligent",
            "overwhelmed",
        ],
    },
    {
        "username": "Giannis",
        "age": 21,
        "sex": "male",
        "education_level": "College",
        "sexual_orientation": "Pansexual",
        "demographic_group": "White",
        "current_employment": "Game Developer",
        "special_instructions": "",
        "personality_characteristics": [
            "strategic",
            "meticulous",
            "nerdy",
            "hyper-focused",
        ],
    },
]
personas = [LLMPersona(**data) for data in persona_data]
actors = [
    LLMActor(
        model=llm,
        name=p.username,
        attributes=p.to_attribute_list(),
        context=CONTEXT,
        instructions=INSTRUCTIONS,
        actor_type=ActorType.USER,
    )
    for p in personas
]
turn_manager = RoundRobbin()
turn_manager.initialize_names([actor.name for actor in actors])

### Running the experiment

In [ ]:
from syndisco.experiments import DiscussionExperiment
from pathlib import Path
import tempfile


disc_exp = DiscussionExperiment(
    topics=[
        "Should programmers be allowed to analyze data?",
        "Should data analysts be allowed to code",
    ],
    users=actors,
    moderator=None,
    num_turns=3,
    num_discussions=2,
)
discussions_dir = Path(tempfile.TemporaryDirectory().name)
disc_exp.begin(discussions_output_dir=discussions_dir)

## Annotation Experiments

In [4]:
from syndisco.backend.model import TransformersModel
from syndisco.backend.persona import LLMPersona
from syndisco.backend.actors import LLMActor, ActorType


# annotator agents are not given a username
# in order to avoid accidental name conflicts with participants
annotator_persona = LLMPersona(
    **{
        "username": "",
        "age": 38,
        "sex": "female",
        "education_level": "Bachelor's",
        "sexual_orientation": "Heterosexual",
        "demographic_group": "White",
        "current_employment": "Annotator",
        "special_instructions": "",
        "personality_characteristics": ["competent"],
    }
)

annotator = LLMActor(
    model=llm,
    name="",
    attributes=annotator_persona.to_attribute_list(),
    context="You are annotating an online discussion",
    instructions="From a scale of 1 (not toxic) to 5 (very toxic) how toxic is the following comment? Reply only with a number.",
    actor_type=ActorType.ANNOTATOR,
)

In [ ]:
from syndisco.experiments import AnnotationExperiment

ann_exp = AnnotationExperiment(annotators=[annotator])
annotations_dir = Path(tempfile.TemporaryDirectory().name)
ann_exp.begin(
    discussions_dir=discussions_dir,
    output_dir=annotations_dir
)